In [22]:
import requests 
from bs4 import BeautifulSoup 
import re
import pandas as pd

In [23]:
#check data exist or not
def ckeck_exist(url,pages):
    for item in pages:
        if url in item:
            return 0

In [24]:
# 發起請求
def start_requests(url):
    #print(url)
    r = requests.get(url)
    return r.content

In [25]:
# 解析一級網頁 獲取 url List
def get_page(text):
    soup = BeautifulSoup(text, 'html.parser')
    movies = soup.find_all('div', class_ = 'release_info')
    pages = []
    for movie in movies:
        url = movie.find('div', class_='release_movie_name').a['href']
        tf = ckeck_exist(url,pages)
        if tf == 0:
            continue
        pages.append(url)
        
    return pages

In [26]:
# 解析二級網頁 獲取訊息
def parse_page(url):
    soup = BeautifulSoup(url, 'html.parser')
    info = {}
    categories = []
    category = {}
    info['中文名稱'] = soup.find('div',class_= 'movie_intro_info_r').find('h1').text.strip()
    info['英文名稱'] = soup.find('div',class_= 'movie_intro_info_r').find('h3').text.strip()
    info['上映日期'] = soup.find('div',class_= 'movie_intro_info_r').find('span').text.split('上映日期：')[1]

    for i,d in enumerate(soup.find('div', class_='level_name_box').find_all('div',class_='level_name')):
        category[i] = d.find('a', class_='gabtn').text.strip()

    categories.append(category)
    info['類別'] = categories
    info['劇情介紹'] = re.sub('\s+','', soup.find('div', class_='gray_infobox_inner').find('span').text.strip())
    return info

In [27]:
result_list = []
for genre in range(1, 22):
    for page in range(1,31):
        url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id={}&page={}'.format(genre,page)
        text = start_requests(url)
        pageurls = get_page(text) # 解析一級頁面
        for pageurl in pageurls: # 解析二級頁面
            page = start_requests(pageurl)
            mydict = parse_page(page)
            result_list.append(mydict)
#write data to csv
df = pd.DataFrame(result_list)
df.to_csv('movies.csv', encoding='utf_8_sig')

df

中文名稱                                       英文名稱        上映日期  \
0   小小兵2：格魯的崛起                  Minions: The Rise of Guru  2022-07-01   
1          蝙蝠俠                                 The Batman  2022-03-04   
2          魔比斯                                    Morbius  2022-01-16   
3     捍衛戰士：獨行俠                          Top Gun: Maverick  2021-11-19   
4     魔鬼剋星：未來世                    Ghostbusters: Afterlife  2021-11-11   
5           沙丘                                       Dune  2021-10-14   
6      007生死交戰                             No Time to Die  2021-10-01   
7      猛毒2：血蜘蛛                Venom: Let There Be Carnage  2021-09-16   
8      尚氣與十環傳奇  Shang-Chi and the Legend of the Ten Rings  2021-09-03   
9         疾凍救援                               The Ice Road  2021-08-25   
10       殺手保鑣2              The Hitman's Wife's Bodyguard  2021-08-25   
11  密弒遊戲2：勝者危亡       Escape Room: Tournament of Champions  2021-08-20   
12   特種部隊：蛇眼之戰                                 Snake Eyes  2021-08-18   
13        脫稿玩家                   Original title: Free Guy  2021-08-12   
14       玩命關頭9                           Fast & Furious 9  2021-08-11   
15    自殺突擊隊：集結                          The Suicide Squad  2021-08-05   
16          除暴                             Caught In Time  2021-07-30   
17        叢林奇航                              Jungle Cruise  2021-07-30   
18       宇宙特攻隊                                  Cosmoball  2021-07-22   
19         黑寡婦                                Black Widow  2021-07-14   

                                           類別  \
0               [{0: '動畫', 1: '動作', 2: '喜劇'}]   
1                        [{0: '動作', 1: '犯罪'}]   
2                        [{0: '動作', 1: '劇情'}]   
3                        [{0: '動作', 1: '劇情'}]   
4   [{0: '動作', 1: '冒險', 2: '懸疑/驚悚', 3: '喜劇'}]   
5               [{0: '動作', 1: '科幻', 2: '劇情'}]   
6            [{0: '動作', 1: '冒險', 2: '懸疑/驚悚'}]   
7                        [{0: '動作', 1: '科幻'}]   
8               [{0: '動作', 1: '冒險', 2: '奇幻'}]   
9                                 [{0: '動作'}]   
10                       [{0: '動作', 1: '喜劇'}]   
11           [{0: '動作', 1: '恐怖', 2: '懸疑/驚悚'}]   
12                       [{0: '動作', 1: '冒險'}]   
13                       [{0: '動作', 1: '喜劇'}]   
14                       [{0: '動作', 1: '冒險'}]   
15                       [{0: '動作', 1: '冒險'}]   
16                       [{0: '動作', 1: '犯罪'}]   
17              [{0: '動作', 1: '冒險', 2: '奇幻'}]   
18                       [{0: '動作', 1: '科幻'}]   
19                       [{0: '動作', 1: '冒險'}]   

                                                 劇情介紹  
0   今年暑假，影史最賣座的動畫電影系列之一和風靡全球的現象，即將帶來關於一個12歲小男孩的故事，...  
1   《蝙蝠俠》將聚焦於一個年輕版本的布魯斯韋恩，並從中檢視他在面罩之下的靈魂。羅伯派汀森將在新版...  
2   ★索尼與漫威繼《猛毒》後再一野心暗黑爭議之作★《蜘蛛人》宇宙正式成立全新神秘反派吸血鬼「魔比...  
3   派拉蒙影業1986年推出的《捍衛戰士》，由湯姆克魯斯主演，是一齣以航空母艦與美國海軍戰鬥機武...  
4   ★抓鬼始祖回歸！《魔鬼剋星未來世》2020年夏天重磅登場★《蟻人》保羅路德攜《怪奇物與》天才...  
5   《沙丘》講述一個神祕又性格強烈的英雄之心路歷程，本片講述保羅亞崔迪的故事，他是一個聰明又富有...  
6   一向被視為年度大片的007系列電影即將推出第25部，丹尼爾克雷格再度飾演所向披靡，家喻戶曉的...  
7   2018年《猛毒》推出後全球累積票房就突破8.5億，創下該年度全球票房第7名的佳績。睽違三年...  
8   漫威電影宇宙再度擴張，全新動作英雄《尚氣與十環傳奇》正式登場，故事敘述尚氣（劉思慕飾演）自幼...  
9   ★今年暑假超級災難動作強檔！★《即刻救援》億萬巨星連恩尼遜年度鉅獻！★耗資4000萬美金拍攝...  
10  ★億萬票房《殺手保鑣》系列續集，暑假必看!★影史最強三"賤"客，原班人馬霸氣回歸，戰力全開「...  
11  為了將害人致死的「世界頂尖密室逃脫」主辦方繩之以法，兩位遊戲生存者柔伊（泰勒羅素飾）和班（羅...  
12  《特種部隊：蛇眼之戰》一片由亨利高汀領銜主演蛇眼，一個充滿決心毅力的獨行俠，在拯救了一個古老...  
13  《脫稿玩家》故事描述一個銀行小職員-蓋伊(萊恩雷諾斯)日復一日年復一年做著無聊的工作，過著樸...  
14  馮迪索飾演的唐老大和莉蒂與他的兒子小布萊恩，正在過著與世隔絕的平靜生活，但是他們都知道這種平...  
15  歡迎來到地獄—全美死亡率最高的「美夢監獄」。那裡關押著最兇惡的超級反派，他們為了逃離該處願意...  
16  ★超級硬派警匪決戰，6年內犯下破萬件搶案的最強悍匪與刑警強力對決★金獎影帝吳彥祖與王千源突破...  
17  故事描述一位名叫霍頓博士（艾蜜莉布朗飾演）發現了深藏在亞馬遜裡，有著一顆擁有治癒神力的奇幻之...  
18  ★俄羅斯影史最強特效場景★《馴龍高手》《變形金剛》億萬動畫團隊再造顛峰★網友狂讚！媲美《艾莉...  
19  漫威影業2020年的首部重頭戲，是粉絲請求多時，終於實現的《黑寡婦》個人電影，黑寡婦在漫威電...